In [1]:
# Install the watermark package.
# This package is used to record the versions of other packages used in this Jupyter notebook.
# https://github.com/rasbt/watermark
!pip install -q -U watermark

In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F 
import torch.optim as optim
from torchsummary import summary

import time
import pandas as pd

In [14]:
# Load the watermark extension to display information about the Python version and installed packages.
%reload_ext watermark

# Display the versions of Python and installed packages.
%watermark -a 'Fabiano Falcão' -ws "https://fabianumfalco.github.io/" --python --iversions

Author: Fabiano Falcão

Website: https://fabianumfalco.github.io/

Python implementation: CPython
Python version       : 3.10.6
IPython version      : 8.11.0

torchvision: 0.15.1
torch      : 2.0.0
pandas     : 1.5.3



In [15]:
# Check if the PyTorch CUDA library is available on the system.
# If it is available, set the device to "cuda", indicating that the GPU will be used.
# Otherwise, set the device to "cpu", indicating that the CPU will be used for computation.
# The chosen device will be used to allocate and execute PyTorch tensors.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Display the selected device
device

device(type='cuda')

In [16]:
# Net class that defines the architecture of a convolutional neural network (CNN)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Definition of the layers of the convolutional neural network
        self.conv1 = nn.Conv2d(1, 6, 5)  # First convolutional layer: input with 1 channel, 6 filters of size 5x5
        self.pool = nn.MaxPool2d(2, 2)  # Pooling layer: performs downsampling with a filter of size 2x2 and stride 2
        self.conv2 = nn.Conv2d(6, 16, 5)  # Second convolutional layer: input with 6 channels, 16 filters of size 5x5
        self.fc1 = nn.Linear(16 * 4 * 4, 120)  # First fully connected (FC) layer: 16 * 4 * 4 inputs, 120 outputs
        self.fc2 = nn.Linear(120, 84)  # Second FC layer: 120 inputs, 84 outputs
        self.fc3 = nn.Linear(84, 10)  # Third FC layer: 84 inputs, 10 outputs (number of classes)

    def forward(self, x):
        # Forward propagation of data through the network layers
        x = self.pool(F.relu(self.conv1(x)))  # Apply the first convolutional layer, followed by a ReLU activation function and pooling
        x = self.pool(F.relu(self.conv2(x)))  # Apply the second convolutional layer, followed by a ReLU activation function and pooling
        x = x.view(-1, 16 * 4 * 4)  # Reshape the tensor to be compatible with the fully connected layer
        x = F.relu(self.fc1(x))  # Apply the first fully connected layer, followed by a ReLU activation function
        x = F.relu(self.fc2(x))  # Apply the second fully connected layer, followed by a ReLU activation function
        x = self.fc3(x)  # Apply the third fully connected layer
        return x

In [17]:
# The method responsible for loading the training and test datasets
def load_dataset(dataset='MNIST', path='./data', batch_size=32, num_workers=1):
    # Define the transformation to be applied to the data
    transform = transforms.Compose(
    [transforms.ToTensor(),  # Convert images to tensors
     transforms.Normalize((0.5,), (0.5,))])  # Normalize images with mean 0.5 and standard deviation 0.5

    if dataset == 'MNIST':
        # Load the MNIST training dataset
        trainset = torchvision.datasets.MNIST(root=path, train=True,
                                            download=True, transform=transform)
        # Load the MNIST test dataset
        testset = torchvision.datasets.MNIST(root=path, train=False,
                                        download=True, transform=transform)
    elif dataset == 'CIFAR10':
        # Load the CIFAR10 training dataset
        trainset = torchvision.datasets.CIFAR10(root=path, train=True,
                                            download=True, transform=transform)
        # Load the CIFAR10 test dataset
        testset = torchvision.datasets.CIFAR10(root=path, train=False,
                                        download=True, transform=transform)
    elif dataset == 'CIFAR100':
        # Load the CIFAR100 training dataset
        trainset = torchvision.datasets.CIFAR100(root=path, train=True,
                                            download=True, transform=transform)
        # Load the CIFAR100 test dataset
        testset = torchvision.datasets.CIFAR100(root=path, train=False,
                                        download=True, transform=transform)
    elif dataset == 'FashionMNIST':
        # Load the FashionMNIST training dataset
        trainset = torchvision.datasets.FashionMNIST(root=path, train=True,
                                            download=True, transform=transform)
        # Load the FashionMNIST test dataset
        testset = torchvision.datasets.FashionMNIST(root=path, train=False,
                                        download=True, transform=transform)
    elif dataset == 'EMNIST':
        # Load the EMNIST training dataset
        trainset = torchvision.datasets.EMNIST(root=path, split='balanced', train=True,
                                            download=True, transform=transform)
        # Load the EMNIST test dataset
        testset = torchvision.datasets.EMNIST(root=path, split='balanced', train=False,
                                        download=True, transform=transform)
    elif dataset == 'KMNIST':
        # Load the KMNIST training dataset
        trainset = torchvision.datasets.KMNIST(root=path, train=True,
                                            download=True, transform=transform)
        # Load the KMNIST test dataset
        testset = torchvision.datasets.KMNIST(root=path, train=False,
                                        download=True, transform=transform)
    elif dataset == 'QMNIST':
        # Load the QMNIST training dataset
        trainset = torchvision.datasets.QMNIST(root=path, what='train',
                                            download=True, transform=transform)
        # Load the QMNIST test dataset
        testset = torchvision.datasets.QMNIST(root=path, what='test',
                                        download=True, transform=transform)
    elif dataset == 'STL10':
        # Load the STL10 training dataset
        trainset = torchvision.datasets.STL10(root=path, split='train',
                                            download=True, transform=transform)
        # Load the STL10 test dataset
        testset = torchvision.datasets.STL10(root=path, split='test',
                                        download=True, transform=transform)
    elif dataset == 'USPS':
        # Load the USPS training dataset
        trainset = torchvision.datasets.USPS(root=path, train=True,
                                            download=True, transform=transform)
        # Load the USPS test dataset
        testset = torchvision.datasets.USPS(root=path, train=False,
                                        download=True, transform=transform)
    else:
        raise ValueError('Invalid dataset. Please choose one of the supported datasets.')

    # Create dataloaders for the training and test datasets
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                            shuffle=True, num_workers=num_workers)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                            shuffle=False, num_workers=num_workers)
    
    # Return the trainloader and testloader
    return trainloader, testloader


In [18]:
training_time_list = []
testing_time_list = []
accuracy_test_list = []
loss_test_list = []
epochs = range(2)

In [19]:
net = Net().to(device)  # Instancia o modelo e o move para a GPU, se disponível

criterion = nn.CrossEntropyLoss().to(device)  # Mova a função de perda para a GPU, se disponível
optimizer = optim.Adam(net.parameters(), lr=0.001)  # Define o otimizador para atualizar os parâmetros do modelo

In [20]:
print('[torch-summary] Model Summary with torchvision.datasets.MNIST')
summary(net, (1, 28, 28))  # Resume o modelo, fornecendo o tamanho de entrada (1 canal, 28x28 pixels)

[torch-summary] Model Summary with torchvision.datasets.MNIST
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 24, 24]             156
         MaxPool2d-2            [-1, 6, 12, 12]               0
            Conv2d-3             [-1, 16, 8, 8]           2,416
         MaxPool2d-4             [-1, 16, 4, 4]               0
            Linear-5                  [-1, 120]          30,840
            Linear-6                   [-1, 84]          10,164
            Linear-7                   [-1, 10]             850
Total params: 44,426
Trainable params: 44,426
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.17
Estimated Total Size (MB): 0.22
----------------------------------------------------------------


In [21]:
trainloader, testloader = load_dataset()  # Carrega os conjuntos de dados de treinamento e teste

loss_list = []

print('Starting training with %d images' % ( len(trainloader.dataset)))

start_time = time.time()
for epoch in epochs:
    running_loss = 0.0  # Variável para armazenar a perda acumulada durante o treinamento
    
   
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Mova os dados para a GPU, se disponível

        optimizer.zero_grad()  # Zera os gradientes dos parâmetros

        outputs = net(inputs)  # Propagação direta dos dados através do modelo
        loss = criterion(outputs, labels)  # Calcula a perda

        loss.backward()  # Retropropagação para calcular os gradientes
        optimizer.step()  # Atualiza os parâmetros do modelo com base nos gradientes

        running_loss += loss.item()  # Acumula a perda para fins de exibição
            
        if i == len(trainloader) - 1:
            last_loss = running_loss /  ((i % 100)+1)
            print('[%d, %5d] Last loss: %.3f' % (epoch + 1, i+1, last_loss  ) )
            loss_list.append(last_loss )  # Armazena o loss médio da época na lista

        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0       

end_time = time.time()
duration = end_time - start_time
training_time_list.append(duration)
# Converte a duração para o formato hh:mm:ss
hours = int(duration // 3600)
minutes = int((duration % 3600) // 60)
seconds = int(duration % 60)

print('Finished Training')
print(f"\nTraining Time: {hours:02d}:{minutes:02d}:{seconds:02d}")
print('Training Time Duration: ',duration)


correct = 0  # Variável para armazenar o número de previsões corretas
total = 0  # Variável para armazenar o número total de exemplos de teste

start_time = time.time()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)  # Mova os dados para a GPU, se disponível

        outputs = net(images)  # Propagação direta dos dados de teste através do modelo
        _, predicted = torch.max(outputs.data, 1)  # Obtém as previsões com maior probabilidade

        total += labels.size(0)  # Atualiza o número total de exemplos de teste
        correct += (predicted == labels).sum().item()  # Conta o número de previsões corretas

end_time = time.time()
duration = end_time - start_time
testing_time_list.append(duration)        
        
accuracy_test = correct / total
loss_test = (total - correct) / total
        
accuracy_test_pct = 100 * accuracy_test
loss_test_pct = 100 * loss_test

accuracy_test_list.append(accuracy_test)
loss_test_list.append(loss_test)

# Converte a duração para o formato hh:mm:ss
hours = int(duration // 3600)
minutes = int((duration % 3600) // 60)
seconds = int(duration % 60)
print(f"\nTesting Time: {hours:02d}:{minutes:02d}:{seconds:02d}")
print('Testing Time Duration: ',duration)

print('\nAccuracy of the network on the %d test images: %.3f' % (len(testloader.dataset), accuracy_test))
print('Loss of the network on the %d test images: %.3f' % (len(testloader.dataset), loss_test))
print('\nAccuracy of the network on the %d test images: %.3f %%' % (len(testloader.dataset), accuracy_test_pct))
print('Loss of the network on the %d test images: %.3f %%' % (total, loss_test_pct))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9912422/9912422 [00:00<00:00, 11489243.34it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 17956669.70it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 7796682.63it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<00:00, 7207918.57it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Starting training with 60000 images


[1,   100] loss: 1.305
[1,   200] loss: 0.478
[1,   300] loss: 0.296
[1,   400] loss: 0.229
[1,   500] loss: 0.159
[1,   600] loss: 0.164
[1,   700] loss: 0.125
[1,   800] loss: 0.131
[1,   900] loss: 0.122
[1,  1000] loss: 0.101
[1,  1100] loss: 0.119
[1,  1200] loss: 0.119
[1,  1300] loss: 0.110
[1,  1400] loss: 0.089
[1,  1500] loss: 0.090
[1,  1600] loss: 0.085
[1,  1700] loss: 0.073
[1,  1800] loss: 0.079
[1,  1875] Last loss: 0.081
[2,   100] loss: 0.073
[2,   200] loss: 0.078
[2,   300] loss: 0.064
[2,   400] loss: 0.067
[2,   500] loss: 0.070
[2,   600] loss: 0.057
[2,   700] loss: 0.067
[2,   800] loss: 0.072
[2,   900] loss: 0.066
[2,  1000] loss: 0.062
[2,  1100] loss: 0.068
[2,  1200] loss: 0.061
[2,  1300] loss: 0.074
[2,  1400] loss: 0.046
[2,  1500] loss: 0.058
[2,  1600] loss: 0.065
[2,  1700] loss: 0.063
[2,  1800] loss: 0.071
[2,  1875] Last loss: 0.057
Finished Training

Training Time: 00:00:26
Training Time Duration:  26.493074655532837

Testing Time: 00:00:02
Testi

In [49]:
def train_and_test(dataset='MNIST'):
    trainloader, testloader = load_dataset(dataset)  # Load the training and test datasets

    loss_list = []

    print('[%s] Starting training with %d images' % (dataset, len(trainloader.dataset)))

    start_time = time.time()
    for epoch in epochs:
        running_loss = 0.0  # Variable to store the accumulated loss during training

        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)  # Move the data to the GPU, if available

            optimizer.zero_grad()  # Zero the gradients of the parameters

            outputs = net(inputs)  # Forward pass the data through the model
            loss = criterion(outputs, labels)  # Calculate the loss

            loss.backward()  # Backpropagation to compute the gradients
            optimizer.step()  # Update the model parameters based on the gradients

            running_loss += loss.item()  # Accumulate the loss for display purposes

            if i == len(trainloader) - 1:
                last_loss = running_loss / ((i % 100) + 1)
                print('[%d, %5d] Last loss: %.3f' % (epoch + 1, i+1, last_loss))
                loss_list.append(last_loss)  # Store the epoch's average loss in the list

            if i % 100 == 99:
                print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                running_loss = 0.0

    end_time = time.time()
    training_time = end_time - start_time
    
    print('[%s] Finished Training' % (dataset))
    
    
    # Convert the duration to the format hh:mm:ss
    #hours = int(training_time // 3600)
    #minutes = int((training_time % 3600) // 60)
    #seconds = int(training_time % 60)

    #print('Finished Training')
    #print(f"\nTraining Time: {hours:02d}:{minutes:02d}:{seconds:02d}")
    #print('Training Time Duration: ', training_time)

    correct = 0  # Variable to store the number of correct predictions
    total = 0  # Variable to store the total number of test examples

    start_time = time.time()
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)  # Move the data to the GPU, if available

            outputs = net(images)  # Forward pass the test data through the model
            _, predicted = torch.max(outputs.data, 1)  # Get the predictions with highest probability

            total += labels.size(0)  # Update the total number of test examples
            correct += (predicted == labels).sum().item()  # Count the number of correct predictions

    end_time = time.time()
    testing_time = end_time - start_time

    accuracy_test = correct / total
    loss_test = (total - correct) / total

    accuracy_test_pct = 100 * accuracy_test
    loss_test_pct = 100 * loss_test

    accuracy_test_list.append(accuracy_test)
    loss_test_list.append(loss_test)


    #print('\nAccuracy of the network on the %d test images: %.3f' % (len(testloader.dataset), accuracy_test))
    #print('Loss of the network on the %d test images: %.3f' % (len(testloader.dataset), loss_test))
    #print('\nAccuracy of the network on the %d test images: %.3f %%' % (len(testloader.dataset), accuracy_test_pct))
    #print('Loss of the network on the %d test images: %.3f %%' % (total, loss_test_pct))
    
    return {
        'loss_list': loss_list,
        'training_time': training_time,
        'testing_time': testing_time,
        'accuracy_test': accuracy_test,
        'loss_test': loss_test
    }

In [63]:
from tqdm import tqdm

def train_and_test_tqdm(dataset='MNIST'):
    trainloader, testloader = load_dataset(dataset)  # Load the training and test datasets

    loss_list = []
    
    print('[%s] %d training images / %d testing images' % (dataset, len(trainloader.dataset),len(testloader.dataset)))
    

    start_time = time.time()
    for epoch in epochs:
        running_loss = 0.0  # Variable to store the accumulated loss during training

        # Use tqdm to create a progress bar for the training loop
        with tqdm(total=len(trainloader), unit='batch', ncols=100, desc="Training") as pbar:
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)  # Move the data to the GPU, if available

                optimizer.zero_grad()  # Zero the gradients of the parameters

                outputs = net(inputs)  # Forward pass the data through the model
                loss = criterion(outputs, labels)  # Calculate the loss

                loss.backward()  # Backpropagation to compute the gradients
                optimizer.step()  # Update the model parameters based on the gradients

                running_loss += loss.item()  # Accumulate the loss for display purposes

                if i == len(trainloader) - 1:
                    last_loss = running_loss / ((i % 100) + 1)
                    #print('[%d, %5d] Last loss: %.3f' % (epoch + 1, i+1, last_loss))
                    loss_list.append(last_loss)  # Store the epoch's average loss in the list

                if i % 100 == 99:
                    #print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
                    running_loss = 0.0

                # Update the tqdm progress bar
                pbar.update(1)

        end_time = time.time()
        training_time = end_time - start_time

        correct = 0  # Variable to store the number of correct predictions
        total = 0  # Variable to store the total number of test examples

        start_time = time.time()
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)  # Move the data to the GPU, if available

                outputs = net(images)  # Forward pass the test data through the model
                _, predicted = torch.max(outputs.data, 1)  # Get the predictions with highest probability

                total += labels.size(0)  # Update the total number of test examples
                correct += (predicted == labels).sum().item()  # Count the number of correct predictions

        end_time = time.time()
        testing_time = end_time - start_time

        accuracy_test = correct / total
        loss_test = (total - correct) / total

        accuracy_test_pct = 100 * accuracy_test
        loss_test_pct = 100 * loss_test

        accuracy_test_list.append(accuracy_test)
        loss_test_list.append(loss_test)

        return {
            'loss_list': loss_list,
            'training_time': training_time,
            'testing_time': testing_time,
            'accuracy_test': accuracy_test,
            'loss_test': loss_test
        }


In [65]:
result_MNIST = train_and_test_tqdm()
result_FashionMNIST = train_and_test_tqdm('FashionMNIST')
#result_EMNIST = train_and_test('EMNIST')
#result_KMNIST = train_and_test('KMNIST')
#result_QMNIST = train_and_test('QMNIST')




[MNIST] 60000 training images / 10000 testing images


Training: 100%|█████████████████████████████████████████████| 1875/1875 [00:13<00:00, 138.89batch/s]


[FashionMNIST] 60000 training images / 10000 testing images


Training: 100%|█████████████████████████████████████████████| 1875/1875 [00:13<00:00, 139.10batch/s]


In [51]:
loss_list = result_MNIST['loss_list']
training_time = result_MNIST['training_time']
testing_time = result_MNIST['testing_time']
accuracy_test = result_MNIST['accuracy_test']
loss_test = result_MNIST['loss_test']

print('\n\nAgora você pode usar os valores retornados da seguinte maneira')
print('Loss list:', loss_list)
print('Training time:', training_time)
print('Testing time:', testing_time)
print('Accuracy on test images:', accuracy_test)
print('Loss on test images:', loss_test)



Agora você pode usar os valores retornados da seguinte maneira
Loss list: [0.01681104075919703, 0.007888377564149777]
Training time: 26.64787006378174
Testing time: 2.280391216278076
Accuracy on test images: 0.9912
Loss on test images: 0.0088


In [40]:
accuracy_test_MNIST

0.9904

In [54]:
epoch_list = [x + 1 for x in list(epochs)]

table_data = {'epoch': epoch_list, 
              'loss_NMIST': result_MNIST['loss_list'], 
              'loss_FashionMNIST': result_FashionMNIST['loss_list']}
df_training = pd.DataFrame(table_data)
df_training.set_index('epoch', inplace=True)
df_training

,loss_NMIST,loss_FashionMNIST
epoch,,
1,0.016811,0.338154
2,0.007888,0.293769


In [23]:
# Manually create the dataset_list
dataset_list = ["NMIST"]
#dataset_list = ["NMIST","CIFAR10","CIFAR100","FashionMNIST","EMNIST","KMNIST","QMNIST","STL10","USPS"] 

# Create a dictionary with the list values
data = {'Dataset': dataset_list,
        'Training Time (s)': training_time_list,
        'Testing Time (s)': testing_time_list,
        'Accuracy Test': accuracy_test_list,
        'Loss Test': loss_test_list}

# Create a DataFrame from the dictionary
df_model_dataset = pd.DataFrame(data)

df_model_dataset.set_index('Dataset', inplace=True)

# Display the DataFrame
df_model_dataset.transpose()

Dataset,NMIST
Training Time (s),26.493075
Testing Time (s),2.255108
Accuracy Test,0.975500
Loss Test,0.024500
